In [ ]:
import pandas as pd

df = pd.read_excel("C:\\Users\\fernando.araujo_logg\\Downloads\\Payables.xlsx")

# Passo 1: Agrupa pelas duas colunas e conta as ocorrências
duplicatas = df.groupby(['Loggi ID', 'Tipo Documento']).size().reset_index(name='Contagem')

# Passo 2: Filtra onde a contagem é maior que 1
duplicatas = duplicatas[duplicatas['Contagem'] > 1]

# Passo 3 (opcional): Recupera as linhas originais que fazem parte dessas duplicatas
df_duplicados = df.merge(duplicatas[['Loggi ID', 'Tipo Documento']], on=['Loggi ID', 'Tipo Documento'], how='inner')

# Exibe o resultado
# display(df_duplicados)

df_duplicados.to_excel("C:\\Users\\fernando.araujo_logg\\Downloads\\duplicidades.xlsx", index=False)

In [ ]:
pip install google-generativeai

In [1]:
import requests
import google.generativeai as genai
from time import sleep as slp
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
import pandas as pd
from pathlib import Path

# Carrega .env
env_path = Path('auth/.env')
load_dotenv(dotenv_path=env_path)

# Configura chave da API do Gemini
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("API KEY do Gemini não encontrada no .env")

genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.0-flash')

# Datas de busca
dtinicio = '20250701'
dtfim = '20250716'

# URL da API
url = f"https://loggi-prod.it-cpi008-rt.cfapps.br10.hana.ondemand.com/http/v1/read/drivtransf?DATE_START={dtinicio}&DATE_END={dtfim}"

# Autenticação da API
username = os.getenv("API_USERNAME")
password = os.getenv("API_PASSWORD")
if not username or not password:
    raise ValueError("Usuário ou senha da API não encontrados no .env")

# Chamada da API
response = requests.get(url, auth=HTTPBasicAuth(username, password))

if response.status_code == 200:
    dados_json = response.json()

    # Transforma em DataFrame
    df = pd.DataFrame(dados_json)

    print("Colunas disponíveis:", df.columns.tolist())  # DEBUG

    # Define colunas desejadas e filtra apenas as que existem
    colunas_desejadas = ['LOGGI_ID', 'DOCTYPE', 'BP_NUMBER', 'AMOUT', 'PERIOD_START', 'PERIOD_END', 'MSG']
    colunas_existentes = [col for col in colunas_desejadas if col in df.columns]

    if not colunas_existentes:
        raise ValueError("Nenhuma das colunas esperadas foi encontrada na resposta da API.")

    df_filtrado = df[colunas_existentes]

    # Geração de sugestões via Gemini
    sugestoes = []

    for i, msg in enumerate(df_filtrado.get('MSG', [])):
        try:
            if not msg:
                sugestoes.append("Mensagem vazia.")
                continue

            resposta = model.generate_content(msg)
            sugestoes.append(resposta.text)
            slp(1)  # delay para evitar rate limit
        except Exception as e:
            sugestoes.append(f"Erro ao processar: {str(e)}")

    df_filtrado['sugestao_gemini'] = sugestoes

    print("\nResultado com sugestões:")
    print(df_filtrado)

else:
    print(f"Erro ao consultar API: {response.status_code}")
    print(response.text)


c:\Python\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: API KEY do Gemini não encontrada no .env